<a href="https://colab.research.google.com/github/jaalvalcan/GEE_index_sets/blob/main/BlueCarbon_EstimationTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap

# 1. FIX: Manually patch the '_credentials' attribute bug
if not hasattr(ee.data, '_credentials'):
    ee.data._credentials = None

# 2. INITIALIZE: Set your project ID
project_id = 'ee-jaalvalcan'
try:
    ee.Initialize(project=project_id)
except Exception:
    ee.Authenticate()
    ee.Initialize(project=project_id)

# 3. DEFINE AOI: St. Jones River, Delaware (from the video)
roi = ee.Geometry.BBox(-75.52, 39.05, -75.40, 39.15)

# Verify by creating the map object (using the project-safe method)
m = geemap.Map(ee_initialize=False)
m.centerObject(roi, 13)
display(m)

In [ ]:
# A. Elevation (NASADEM)
# Note: Delaware is low-lying, so elevation is the primary driver of marsh type
elevation = ee.Image("NASA/NASADEM_HGT/001").select('elevation').clip(roi)

# B. Distance to Water (Calculated from Global Surface Water)
# Marshes closer to channels often have higher organic matter influx
water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select('occurrence').gt(50)
distance_to_water = water.fastDistanceTransform().sqrt() \
    .multiply(ee.Image.pixelArea().sqrt()).rename('distance')

# C. NDVI (Sentinel-2 2022 Median)
# Represents plant biomass/productivity
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(roi) \
    .filterDate('2022-01-01', '2022-12-31') \
    .median()

ndvi = s2.normalizedDifference(['B8', 'B4']).rename('ndvi').clip(roi)

# Add layers to map to verify coverage
m.addLayer(elevation, {'min': 0, 'max': 5}, 'Elevation (m)')
m.addLayer(ndvi, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, 'NDVI')

In [ ]:
# 1. LOAD TARGET: GEDI L4B (Mean Biomass in Mg/ha)
# This serves as our 'Ground Truth' for the regression
target = ee.Image("LARSE/GEDI/GEDI04_B_002").select('MU').rename('carbon').clip(roi)

# 2. CREATE MANGROVE/MARSH MASK
# We only want to train the model on wetlands
mask = ee.ImageCollection("ESA/WorldCover/v100").first().eq(90).clip(roi)

# 3. STACK EVERYTHING
# The 'constant' band acts as the Y-intercept for our linear equation
stack = ee.Image.cat([
    ee.Image(1).rename('constant'),
    elevation,
    distance_to_water,
    ndvi,
    target
]).updateMask(mask)

In [ ]:
# --- STEP 4 (Corrected) ---
# 1. Sample the stack (1500 points)
training_data = stack.sample(region=roi, scale=100, numPixels=1500, seed=42)

# 2. Calculate Linear Regression
regression = training_data.reduceColumns(
    ee.Reducer.linearRegression(numX=4, numY=1),
    ['constant', 'elevation', 'distance', 'ndvi', 'carbon']
)

# 3. FIX: Convert to Array, project to 1D, then convert to a List for easy access
coeffs = ee.Array(regression.get('coefficients')).project([0]).toList()

# Let's print them to see the relationship
print('Regression Coefficients (Intercept, Elev, Dist, NDVI):', coeffs.getInfo())

# --- STEP 5 ---
# Apply the linear equation: Carbon = c0 + (c1*Elev) + (c2*Dist) + (c3*NDVI)
prediction = ee.Image(1).multiply(ee.Number(coeffs.get(0))) \
    .add(elevation.multiply(ee.Number(coeffs.get(1)))) \
    .add(distance_to_water.multiply(ee.Number(coeffs.get(2)))) \
    .add(ndvi.multiply(ee.Number(coeffs.get(3)))) \
    .rename('Predicted_Blue_Carbon') \
    .updateMask(mask)

# Visualize the result
carbon_viz = {'min': 50, 'max': 450, 'palette': ['#ffffcc', '#78c679', '#006837']}
m.addLayer(prediction, carbon_viz, 'Predicted Blue Carbon (Mg C/ha)')
m.add_colorbar(carbon_viz, label="Carbon Stock (Mg C / ha)")

print("Map generated. Check the map view for the results!")

In [ ]:
"""
Blue Carbon Estimation Tool
Methodology: Warner, D., Callahan, J., McKenna, T. (Delaware Geological Survey)
Implementation: [Javier Valdes]
Platform: Google Earth Engine
Data Sources: NASA GEDI L4B, Sentinel-2, ESA WorldCover
"""